In [3]:
import pandas as pd
import numpy as np
import os
import sys

sys.path.append(os.path.abspath('../src'))
from model_training import get_recommendations, get_content_recommendations_pca, get_sentiment_recommendations, get_recommendations_with_pca, get_review_recommendations, get_topic_recommendations, get_reviewer_overlap_recommendations, get_weighted_hybrid_recommendations, load_all_models, get_content_recommendations, svd_product_recommendations, get_knn_recommendations, prepare_features_for_knn
# Load the DataFrame and set it in the ml module

df = pd.read_parquet('../app/dataset/product_features.parquet')
#print(df.head())
load_all_models()

Recommendation function 'get_recommendations' updated to use reduced feature set.
All models and data loaded successfully.


,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B008IFXQFU,"TP-Link USB WiFi Adapter for PC(TL-WN725N), N1...",Computers&Accessories|NetworkingDevices|Networ...,499.0,999.0,50,4.2,"1,79,691",USB WiFi Adapter —— Speedy wireless transmissi...,"AGV3IEFANZCKECFGUM42MRH5FNOA,AEBO7NWCNXKT4AESA...","Azhar JuMan,Anirudh Sood,Hari Krishnan PS,Akas...","R1LW6NWSVTVZ2H,R3VR5WFKUS15C5,R2F6GC79OYWUKQ,R...",Works on linux for me. Get the model with ante...,I use this to connect an old PC to internet. I...,https://m.media-amazon.com/images/I/31e6ElWRym...,https://www.amazon.in/TP-Link-TL-WN725N-150Mbp...
1,B0B6F7LX4C,MI 80 cm (32 inches) 5A Series HD Ready Smart ...,"Electronics|HomeTheater,TV&Video|Televisions|S...",13999.0,24999.0,44,4.2,"32,840","Note : The brands, Mi and Xiaomi, are part of ...","AHEVOQADJSSRX7DS325HSFLMP7VQ,AG7XYZRCSKX6G2OLO...","Manoj maddheshiya,Manoj Kumar Sahoo,Saumil s.,...","R13UTIA6KOF6QV,R2UGDZSGFF01K7,RHHIZ45VYU5X6,R1...",It is the best tv if you are getting it in 10-...,Pros- xiomi 5a is best in budget-Nice picture ...,https://m.media-amazon.com/images/I/51fmHk3km+...,https://www.amazon.in/MI-inches-Ready-Android-...
2,B0B1YVCJ2Y,Acer 80 cm (32 inches) I Series HD Ready Andro...,"Electronics|HomeTheater,TV&Video|Televisions|S...",11499.0,19990.0,42,4.3,"4,703",Resolution : HD Ready (1366x768) | Refresh Rat...,"AFSMISGEYDYIP3Z42UTQU4AKOYZQ,AF5ILQY4KFDTO5XHH...","Ayush,ROHIT A.,Kedar,Haran,Santosh Ghante,KRIS...","R1EBS3566VCSCG,R24MB66WRPSN2A,R25UU2M1B9BO5X,R...",Wonderful TV and Awful installation service fr...,About the TV - Wonderful----------------------...,https://m.media-amazon.com/images/I/41gikeSuhA...,https://www.amazon.in/Acer-inches-Ready-Androi...
3,B08B42LWKN,OnePlus 80 cm (32 inches) Y Series HD Ready LE...,"Electronics|HomeTheater,TV&Video|Televisions|S...",14999.0,19999.0,25,4.2,"34,899",Resolution: HD Ready (1366x768) | Refresh Rate...,"AFUT7ANZTZYGLXU65EQ2D5OP6UMA,AGT7YYJVUC6ZHRKQH...","ATHARVA BONDRE,Sujitkumar Thorat,Chandrasekara...","R3COVVOP2R7Z28,R2T6WHEO2ONNDD,RUFFV2QR43OCM,R2...","Worthy and most affordable - Great TV,Good pro...",This OnePlus TV is great if you want a afforda...,https://m.media-amazon.com/images/I/41WE9ZGEC4...,https://www.amazon.in/OnePlus-inches-Ready-And...
4,B0B3MMYHYW,OnePlus 126 cm (50 inches) Y Series 4K Ultra H...,"Electronics|HomeTheater,TV&Video|Televisions|S...",32999.0,45999.0,28,4.2,"7,298",Resolution: 4K Ultra HD (3840x2160) | Bezel-le...,"AGDOVGWZKEQ3M6DA2GHV6WUZT5SA,AGEUXHN7U2Q26CM6T...","Abhishek Kumar,Arun prasath,Naveen,Harmit,Plac...","R2J3Q3BUHJ2S7E,R2H2ELE1DG24VY,R1U1S7X7BPSZBU,R...","Decent product. Value for money.,Value for mon...",I am posting this after a few weeks of using t...,https://m.media-amazon.com/images/I/51hQfTroMz...,https://www.amazon.in/OnePlus-50-inches-Androi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,379.0,919.0,59,4.0,"1,090",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...","Prabha ds,Raghuram bk,Real Deal,Amazon Custome...","R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...","Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",https://m.media-amazon.com/images/I/41fDdRtjfx...,https://www.amazon.in/Noir-Aqua-Spanner-Purifi...
1347,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,2280.0,3045.0,25,4.1,"4,118","230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...","Manu Bhai,Naveenpittu,Evatira Sangma,JAGANNADH...","R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...","ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned 

This code performs an Ablation Stability Analysis. It is a validation step used to compare your original "heavy" model (the 10-model ensemble) against the new, streamlined "Core 4" model to see how much the actual recommendations changed.

top_5_uniform (The Baseline): This runs the 10-model version where every model has an equal vote (uniform weighting). This represents the "old" system.

top_5_optimized (The Core 4): This runs the new system, using only the 4 selected models and your calculated optimal weights.

check_ablation_stability - Out of the 5 products we recommended before, how many are still present in the new list?

In [9]:
import pandas as pd

# 1. First, generate the actual recommendations for a specific product
target_pid = "B002PD61Y4" # Example Product ID from your dataset
model_keys = [
            'pca_features',  
            'content_pca', 
            'review_text',  
            'svd_collaborative_optimized'
        ]

core_4_weights = {
        'content_pca': 0.0424,
        'review_text': 0.4764,
        'pca_features': 0.2059,
        'svd_collaborative_optimized': 0.2753
    }

# Generate Baseline (Uniform/10-Model) 
# Note: Ensure your 'get_weighted_hybrid_recommendations' function is defined
top_5_uniform = get_weighted_hybrid_recommendations(df,target_pid, N=5)['product_id'].tolist()

# Generate Optimized (Core 4)
# Note: Ensure you have your 'optimal_weights' defined for the Core 4
top_5_optimized = get_weighted_hybrid_recommendations(df,target_pid, N=5, weights=core_4_weights, model_keys=model_keys)['product_id'].tolist()

# 2. Define the Stability Function
def check_ablation_stability(uniform_top_5, optimized_top_5):
    """
    Analyzes the overlap between a baseline (Uniform) and optimized model.
    """
    set_uniform = set(uniform_top_5)
    set_optimized = set(optimized_top_5)
    
    # Calculate intersection
    overlap = len(set_uniform.intersection(set_optimized))
    return overlap

# 3. Now run the stability check
stability_score = check_ablation_stability(top_5_uniform, top_5_optimized)

print(f"Baseline IDs:  {top_5_uniform}")
print(f"Optimized IDs: {top_5_optimized}")
print(f"---")
print(f"Stability Score: {stability_score}/5")

Baseline IDs:  ['B07CWNJLPC', 'B09LHXNZLR', 'B0141EZMAI', 'B07P681N66', 'B01N6LU1VF']
Optimized IDs: ['B09NNHFSSF', 'B07WGMMQGP', 'B07WHQBZLS', 'B07WJWRNVK', 'B09LHXNZLR']
---
Stability Score: 1/5


1/5 stability, you have proven that the Core 4 is less dependent on popularity.